In [6]:
from pathlib import Path
from datetime import timedelta, datetime
from TickStream import TickStream
from TickResampler import TickResampler
from BarCollector import BarCollector
from backtesting import Backtest, Strategy

In [2]:
Path.cwd()

WindowsPath('e:/GithubRepos/TickPerfectTraversal/services/backtesting_engine')

In [9]:
root = Path("test_data")  # contains 202405/
timeframe = timedelta(seconds=5)

tick_stream = TickStream(root=root, start=datetime(2024, 5, 1, 0, 0), end=datetime(2024, 5, 1, 23, 0))
resampler = TickResampler(timeframe)
collector = BarCollector()

for tick in tick_stream.stream():
    bar = resampler.update(tick)
    if bar is not None:
        collector.add_bar(bar)

df = collector.to_dataframe()
print(df.head())
print(df.tail())

                         Open      High       Low     Close  Volume
Timestamp                                                          
2024-05-01 00:00:00  17526.00  17526.75  17525.75  17525.75       6
2024-05-01 00:00:05  17526.50  17526.50  17526.50  17526.50       1
2024-05-01 00:00:10  17526.25  17526.25  17526.00  17526.00       2
2024-05-01 00:00:15  17527.00  17527.25  17526.75  17527.25       4
2024-05-01 00:00:25  17527.00  17529.25  17527.00  17528.00      68
                         Open      High       Low     Close  Volume
Timestamp                                                          
2024-05-01 21:59:30  17539.00  17539.50  17538.50  17539.50      17
2024-05-01 21:59:35  17539.75  17540.25  17539.75  17540.25      60
2024-05-01 21:59:40  17540.00  17541.50  17540.00  17540.50      33
2024-05-01 21:59:45  17541.50  17541.75  17540.75  17541.50       8
2024-05-01 21:59:50  17540.75  17540.75  17540.75  17540.75       7


In [4]:
class BuyAndHold(Strategy):
    def init(self):
        pass

    def next(self):
        if not self.position:
            self.buy()

In [5]:
bt = Backtest(
    df,
    BuyAndHold,
    cash=100_000,
    commission=0.0,
    finalize_trades=True
)

stats = bt.run()
print(stats)

Start                     2024-05-01 00:00:00
End                       2024-05-09 22:58:00
Duration                      8 days 22:58:00
Exposure Time [%]                    99.97916
Equity Final [$]                     103452.5
Equity Peak [$]                     103671.25
Return [%]                             3.4525
Buy & Hold Return [%]                 3.93947
Return (Ann.) [%]                   150.82961
Volatility (Ann.) [%]                23.15728
CAGR [%]                            159.85542
Sharpe Ratio                          6.51327
Sortino Ratio                       138.24282
Calmar Ratio                         81.02391
Alpha [%]                            -0.00561
Beta                                  0.87781
Max. Drawdown [%]                    -1.86154
Avg. Drawdown [%]                    -0.12378
Max. Drawdown Duration        2 days 09:26:00
Avg. Drawdown Duration        0 days 02:20:00
# Trades                                    1
Win Rate [%]                      

In [11]:
stats.index

Index(['Start', 'End', 'Duration', 'Exposure Time [%]', 'Equity Final [$]',
       'Equity Peak [$]', 'Return [%]', 'Buy & Hold Return [%]',
       'Return (Ann.) [%]', 'Volatility (Ann.) [%]', 'CAGR [%]',
       'Sharpe Ratio', 'Sortino Ratio', 'Calmar Ratio', 'Alpha [%]', 'Beta',
       'Max. Drawdown [%]', 'Avg. Drawdown [%]', 'Max. Drawdown Duration',
       'Avg. Drawdown Duration', '# Trades', 'Win Rate [%]', 'Best Trade [%]',
       'Worst Trade [%]', 'Avg. Trade [%]', 'Max. Trade Duration',
       'Avg. Trade Duration', 'Profit Factor', 'Expectancy [%]', 'SQN',
       'Kelly Criterion', '_strategy', '_equity_curve', '_trades'],
      dtype='object')